<a href="https://colab.research.google.com/github/AffanA2003/X-ray-Dental-Detection/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [55]:
csv_path = 'dataset.csv'
df = pd.read_csv(csv_path)

In [78]:
# Create lists to store image paths and corresponding labels
img_dir = '/images/'
image_paths = []
labels = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    img_filename = row['filename']
    status = row['region_attributes']
    
    # Create the full path to the image file
    img_path = os.path.join(img_dir, img_filename)
    
    # Add image path and label to the lists
    image_paths.append(img_path)
    labels.append(status)

# Convert labels to numerical values
label_mapping = {'{"status":"1"}': 0, '{"status":"2"}':1, '{"status":"3"}': 2}
labels = [label_mapping[label] for label in labels]

# Convert image_paths and labels to numpy arrays
image_paths = np.array(image_paths)
labels = np.array(labels)

# Shuffle the data
image_paths, labels = shuffle(image_paths, labels, random_state=42)

# Split the data into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)


In [82]:
def load_and_preprocess_image(image_path):
    # Load the image using PIL
    img = Image.open(image_path)
    
    # Preprocess the image (e.g., resize, normalize)
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    
    return img

# Load and preprocess training images

train_images = np.array([load_and_preprocess_image(img_path) for img_path in train_image_paths])


test_images = np.array([load_and_preprocess_image(img_path) for img_path in test_image_paths])


In [83]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 3 units for the 3 classes
predictions = Dense(3, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [86]:
# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.0138 - accuracy: 0.5000 - val_loss: 5.9605e-07 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 7.3363 - accuracy: 0.5000 - val_loss: 21.6871 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 11.0591 - accuracy: 0.5000 - val_loss: 1.2550 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.8709 - accuracy: 0.5000 - val_loss: 1.0722 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 0.8464 - accuracy: 0.5000 - val_loss: 0.8252 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 0.7278 - accuracy: 0.5000 - val_loss: 0.1554 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 1.0703 - accuracy: 0.5000 - val_loss: 2.4766 - val_accuracy: 0.0000e+00
Epoch 8/10
1/1 

In [88]:
# Assuming you have extracted the test labels as a list of strings
test_labels = ['0', '1']  # Replace with your actual test labels

# Convert the test labels to a numpy array of integers
test_labels = np.array(test_labels, dtype=np.int32)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


1/1 [==============================] - 1s 1s/step - loss: 0.7358 - accuracy: 0.5000
Test Loss: 0.7358355522155762
Test Accuracy: 0.5
